In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path='drive/My Drive/satilite fire detection/satillite_data/Training'


In [ ]:
batch_siz=32
image_height=128
img_width=128


In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
path,
validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(image_height,img_width),
    batch_size=batch_siz

)

In [ ]:
print(train_ds)
print(train_ds.class_names)

In [ ]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
path,
validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(image_height,img_width),
    batch_size=batch_siz
)

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
train_ds=train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE
)

In [ ]:
num_classes=2
model=tf.keras.models.Sequential([

                                  tf.keras.layers.InputLayer(input_shape=(image_height,img_width,3)),
                                  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),

                                  tf.keras.layers.Conv2D(16,3 ,activation='relu'),
                                  tf.keras.layers.MaxPooling2D(),

                                  tf.keras.layers.Conv2D(32,3,activation='relu'),
                                  tf.keras.layers.MaxPooling2D(),

                                  tf.keras.layers.Conv2D(64,3,activation='relu'),
                                  tf.keras.layers.MaxPooling2D(),


                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(128,activation='relu'),
                                  tf.keras.layers.Dropout(0.5),
                                  tf.keras.layers.Dense(num_classes,activation='softmax')

                                 ])

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

early_stop_back=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=0.001,
                                               patience=5)

history=model.fit(train_ds,validation_data=val_ds,
                 epochs=15)



In [ ]:
model.save("drive/My Drive/satilite fire detection/SatilliteDetector.h5")

In [ ]:
loss,acc=model.evaluate(val_ds,batch_size=batch_siz)
print("Accuracy",acc)
print("loss",loss)

In [ ]:
def val(x):
    global y
    y=x+1
    return y



In [ ]:

def prediction(img):
    class_names = ['fire', 'not fire']
    my_image = load_img(img, target_size=(image_height, img_width))
    my_image = img_to_array(my_image)
    my_image = np.expand_dims(my_image, 0)
    out=np.round(model.predict(my_image)[0], 2)
    fig = plt.figure(figsize=(7, 4))
    plt.barh(class_names, out, color='lightgray',edgecolor='red', linewidth=1,height=0.5)
    for index, value in enumerate(out):
        plt.text(value/2 + 0.1, index, f"{100*value:.2f}%", fontweight='bold')
        plt.xticks([])
        plt.yticks([0, 1], labels=class_names, fontweight='bold',fontsize=14)
        fig.savefig('pred_img.png', bbox_inches='tight')
    if out[0]>out[1]:
        val(1)
        print("Fire detected")
    elif out[0]<out[1]:
        print("No fire detected")
    return plt.show()


In [ ]:
testim="/content/fire10.jpg"
prediction(testim)